## 라이브러리 import

In [ ]:
import pandas            as pd
import numpy             as np
import seaborn           as sb
import scipy.stats       as sp
import matplotlib.pyplot as plt
import tensorflow        as tf
import pywt

### 학습용 데이터

## Time Domain 특징값 추출 (10 features * 3 sensors = 30개씩)

In [ ]:
def rms(x): # RMS 함수 정의
    return np.sqrt(np.mean(x**2))

In [ ]:
NoOfData    = 180
NoOfSensor  = 3    # 가속도(Acceleration), 전압(Voltage), 전류(Current)
NoOfFeature = 10   # 특징 개수:10개 (순서: Max, Min, Mean, RMS, Variance, Skewness, Kurtosis, Crest factor, Impulse factor, Shape factor)

TimeFeature_Normal   = np.zeros((NoOfSensor*NoOfFeature , NoOfData))
TimeFeature_Abnormal1 = np.zeros((NoOfSensor*NoOfFeature , NoOfData))
TimeFeature_Abnormal2 = np.zeros((NoOfSensor*NoOfFeature , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './Train_Data/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './Train_Data/Abnormal1_%d'%(i+1) # Abnormal 데이터 파일 경로
    temp_path3 = './Train_Data/Abnormal2_%d'%(i+1) # Abnormal 데이터 파일 경로
    temp_data1 = pd.read_csv(temp_path1 , sep=',' , header=None) # 임시 Normal 데이터
    temp_data2 = pd.read_csv(temp_path2 , sep=',' , header=None) # 임시 Abnormal 데이터
    temp_data3 = pd.read_csv(temp_path3 , sep=',' , header=None) # 임시 Abnormal 데이터
    
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Normal Time Domain Feature
        TimeFeature_Normal[10*j+0, i] = np.max(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+1, i] = np.min(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+2, i] = np.mean(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+3, i] = rms(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+4, i] = np.var(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+5, i] = sp.skew(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+6, i] = sp.kurtosis(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+7, i] = np.max(temp_data1.iloc[:,j])/rms(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+8, i] = rms(temp_data1.iloc[:,j])/np.mean(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+9, i] = np.max(temp_data1.iloc[:,j])/np.mean(temp_data1.iloc[:,j])
        
        # Abnormal1 Time Domain Feature
        TimeFeature_Abnormal1[10*j+0, i] = np.max(temp_data2.iloc[:,j])
        TimeFeature_Abnormal1[10*j+1, i] = np.min(temp_data2.iloc[:,j])
        TimeFeature_Abnormal1[10*j+2, i] = np.mean(temp_data2.iloc[:,j])
        TimeFeature_Abnormal1[10*j+3, i] = rms(temp_data2.iloc[:,j])
        TimeFeature_Abnormal1[10*j+4, i] = np.var(temp_data2.iloc[:,j])
        TimeFeature_Abnormal1[10*j+5, i] = sp.skew(temp_data2.iloc[:,j])
        TimeFeature_Abnormal1[10*j+6, i] = sp.kurtosis(temp_data2.iloc[:,j])
        TimeFeature_Abnormal1[10*j+7, i] = np.max(temp_data2.iloc[:,j])/rms(temp_data2.iloc[:,j])
        TimeFeature_Abnormal1[10*j+8, i] = rms(temp_data2.iloc[:,j])/np.mean(temp_data2.iloc[:,j])
        TimeFeature_Abnormal1[10*j+9, i] = np.max(temp_data2.iloc[:,j])/np.mean(temp_data2.iloc[:,j])
        
        # Abnormal2 Time Domain Feature
        TimeFeature_Abnormal2[10*j+0, i] = np.max(temp_data2.iloc[:,j])
        TimeFeature_Abnormal2[10*j+1, i] = np.min(temp_data2.iloc[:,j])
        TimeFeature_Abnormal2[10*j+2, i] = np.mean(temp_data2.iloc[:,j])
        TimeFeature_Abnormal2[10*j+3, i] = rms(temp_data2.iloc[:,j])
        TimeFeature_Abnormal2[10*j+4, i] = np.var(temp_data2.iloc[:,j])
        TimeFeature_Abnormal2[10*j+5, i] = sp.skew(temp_data2.iloc[:,j])
        TimeFeature_Abnormal2[10*j+6, i] = sp.kurtosis(temp_data2.iloc[:,j])
        TimeFeature_Abnormal2[10*j+7, i] = np.max(temp_data2.iloc[:,j])/rms(temp_data2.iloc[:,j])
        TimeFeature_Abnormal2[10*j+8, i] = rms(temp_data2.iloc[:,j])/np.mean(temp_data2.iloc[:,j])
        TimeFeature_Abnormal2[10*j+9, i] = np.max(temp_data2.iloc[:,j])/np.mean(temp_data2.iloc[:,j])
        
print(TimeFeature_Normal.shape)
print(TimeFeature_Abnormal1.shape)
print(TimeFeature_Abnormal2.shape)

## Frequency Domain 특징값 추출 (10 features \* 8 wavelet levels * 3 sensors = 240개씩)

In [ ]:
# Wavelet options

MotherWavelet = pywt.Wavelet('haar')   # Mother wavelet (모함수) 지정
Level   = 8                            # Wavelet 분해 레벨 지정
select  = 8                            # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)

In [ ]:
#Frequency Domain 특징값 추출 (Wavelet Transform 기반)
FreqFeature_Normal   = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))
FreqFeature_Abnormal1 = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))
FreqFeature_Abnormal2 = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './Train_Data/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './Train_Data/Abnormal1_%d'%(i+1) # Abnormal1 데이터 파일 경로
    temp_path3 = './Train_Data/Abnormal2_%d'%(i+1) # Abnormal2 데이터 파일 경로
    temp_data1 = pd.read_csv(temp_path1 , sep=',' , header=None) # 임시 Normal 데이터
    temp_data2 = pd.read_csv(temp_path2 , sep=',' , header=None) # 임시 Abnormal1 데이터
    temp_data3 = pd.read_csv(temp_path3 , sep=',' , header=None) # 임시 Abnormal2 데이터
    Coef1      = pywt.wavedec(temp_data1, MotherWavelet, level=Level, axis=0)
    Coef2      = pywt.wavedec(temp_data2, MotherWavelet, level=Level, axis=0)
    Coef3      = pywt.wavedec(temp_data3, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in range(NoOfSensor):
        
        for k in np.arange(select):
            coef1 = Coef1[Level-k]
            coef2 = Coef2[Level-k]
            coef3 = Coef3[Level-k]
            
            # Normal Frequency Domain Feature
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef1[:,j])/np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef1[:,j])/np.mean(coef1[:,j])
            
            # Abnormal1 Frequency Domain Feature
            FreqFeature_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef2[:,j])
            FreqFeature_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef2[:,j])
            FreqFeature_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef2[:,j])
            FreqFeature_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef2[:,j])
            FreqFeature_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef2[:,j])
            FreqFeature_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef2[:,j])
            FreqFeature_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef2[:,j])
            FreqFeature_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef2[:,j])/rms(coef2[:,j])
            FreqFeature_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef2[:,j])/np.mean(coef2[:,j])
            FreqFeature_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef2[:,j])/np.mean(coef2[:,j])
            
            # Abnormal2 Frequency Domain Feature
            FreqFeature_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef3[:,j])
            FreqFeature_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef3[:,j])
            FreqFeature_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef3[:,j])
            FreqFeature_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef3[:,j])
            FreqFeature_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef3[:,j])
            FreqFeature_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef3[:,j])
            FreqFeature_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef3[:,j])
            FreqFeature_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef3[:,j])/rms(coef3[:,j])
            FreqFeature_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef3[:,j])/np.mean(coef3[:,j])
            FreqFeature_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef3[:,j])/np.mean(coef3[:,j])

print(FreqFeature_Normal.shape)
print(FreqFeature_Abnormal1.shape)
print(FreqFeature_Abnormal2.shape)

## Feature 통합 (Time domain 30개 + Freq. domain 240 = 270개)

In [ ]:
Feature_Normal = np.concatenate([TimeFeature_Normal,FreqFeature_Normal] , axis=0)
Feature_Abnormal1 = np.concatenate([TimeFeature_Abnormal1,FreqFeature_Abnormal1] , axis=0)
Feature_Abnormal2 = np.concatenate([TimeFeature_Abnormal2,FreqFeature_Abnormal2] , axis=0)
Normal_FeatureData = pd.DataFrame(Feature_Normal)
Abnormal1_FeatureData = pd.DataFrame(Feature_Abnormal1)
Abnormal2_FeatureData = pd.DataFrame(Feature_Abnormal2)

## P-value 계산

In [ ]:
NoOfFeature = Feature_Normal.shape[0] # 추출된 Feature 갯수

P_value = np.zeros((NoOfFeature , 2))

# 특징값 각각 Anova검정 수행
for i in np.arange(NoOfFeature):
    
    Anova_test       = np.array(sp.f_oneway(Normal_FeatureData.iloc[i,:] , Abnormal1_FeatureData.iloc[i,:] , Abnormal2_FeatureData.iloc[i,:]))
    P_value[i,0] = i          # Feature Index
    P_value[i,1] = Anova_test[1]  # P값 (P-value)
    
P_value      = pd.DataFrame(P_value)
P_value

In [ ]:
P_value_Rank = P_value.sort_values([1],ascending=True)  # P-value 기준 오름차순 정렬

P_value_Rank

In [ ]:
path = './Data/P_value_Rank'   # path = '파일 경로/저장할 파일 이름'
P_value_Rank.to_csv(path, sep=',', header=None, index=None)

## P value 기준 차원축소 대상 데이터(Feature data) 선정

In [ ]:
Number    = 20

Normal_SelectedFeatues = np.zeros((Number,NoOfData))
Abnormal1_SelectedFeatues = np.zeros((Number,NoOfData))
Abnormal2_SelectedFeatues = np.zeros((Number,NoOfData))

for i in range(Number):
    
    index                 = int(P_value_Rank.iloc[i,0])
    Normal_SelectedFeatues[i,:]   = Normal_FeatureData.iloc[index,:].values
    Abnormal1_SelectedFeatues[i,:] = Abnormal1_FeatureData.iloc[index,:].values
    Abnormal2_SelectedFeatues[i,:] = Abnormal2_FeatureData.iloc[index,:].values

# 정상, 고장 특징값 합치기    
FeatureSelected = pd.DataFrame(np.concatenate([Normal_SelectedFeatues, Abnormal1_SelectedFeatues, Abnormal2_SelectedFeatues] , axis=1))
FeatureSelected.shape

### 평가용 데이터

In [ ]:
NoOfTestData    = 20
NoOfSensor  = 3    # 가속도(Acceleration), 전압(Voltage), 전류(Current)
NoOfFeature = 10   # 특징 개수:10개 (순서: Max, Min, Mean, RMS, Variance, Skewness, Kurtosis, Crest factor, Impulse factor, Shape factor)

TimeFeatureTest_Normal   = np.zeros((NoOfSensor*NoOfFeature , NoOfTestData))
TimeFeatureTest_Abnormal1 = np.zeros((NoOfSensor*NoOfFeature , NoOfTestData))
TimeFeatureTest_Abnormal2 = np.zeros((NoOfSensor*NoOfFeature , NoOfTestData))

for i in range(NoOfTestData):
    
    # 데이터 불러오기
    temp_test_path1 = './Test_Data/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_test_path2 = './Test_Data/Abnormal1_%d'%(i+1) # Abnormal1 데이터 파일 경로
    temp_test_path3 = './Test_Data/Abnormal2_%d'%(i+1) # Abnormal2 데이터 파일 경로
    temp_test_data1 = pd.read_csv(temp_test_path1 , sep=',' , header=None) # 임시 Normal 데이터
    temp_test_data2 = pd.read_csv(temp_test_path2 , sep=',' , header=None) # 임시 Abnormal 데이터
    temp_test_data3 = pd.read_csv(temp_test_path3 , sep=',' , header=None) # 임시 Abnormal 데이터
    
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Normal Time Domain Feature
        TimeFeatureTest_Normal[10*j+0, i] = np.max(temp_test_data1.iloc[:,j])
        TimeFeatureTest_Normal[10*j+1, i] = np.min(temp_test_data1.iloc[:,j])
        TimeFeatureTest_Normal[10*j+2, i] = np.mean(temp_test_data1.iloc[:,j])
        TimeFeatureTest_Normal[10*j+3, i] = rms(temp_test_data1.iloc[:,j])
        TimeFeatureTest_Normal[10*j+4, i] = np.var(temp_test_data1.iloc[:,j])
        TimeFeatureTest_Normal[10*j+5, i] = sp.skew(temp_test_data1.iloc[:,j])
        TimeFeatureTest_Normal[10*j+6, i] = sp.kurtosis(temp_test_data1.iloc[:,j])
        TimeFeatureTest_Normal[10*j+7, i] = np.max(temp_test_data1.iloc[:,j])/rms(temp_test_data1.iloc[:,j])
        TimeFeatureTest_Normal[10*j+8, i] = rms(temp_test_data1.iloc[:,j])/np.mean(temp_test_data1.iloc[:,j])
        TimeFeatureTest_Normal[10*j+9, i] = np.max(temp_test_data1.iloc[:,j])/np.mean(temp_test_data1.iloc[:,j])
        
        # Abnormal1 Time Domain Feature
        TimeFeatureTest_Abnormal1[10*j+0, i] = np.max(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal1[10*j+1, i] = np.min(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal1[10*j+2, i] = np.mean(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal1[10*j+3, i] = rms(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal1[10*j+4, i] = np.var(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal1[10*j+5, i] = sp.skew(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal1[10*j+6, i] = sp.kurtosis(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal1[10*j+7, i] = np.max(temp_test_data2.iloc[:,j])/rms(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal1[10*j+8, i] = rms(temp_test_data2.iloc[:,j])/np.mean(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal1[10*j+9, i] = np.max(temp_test_data2.iloc[:,j])/np.mean(temp_test_data2.iloc[:,j])
        
        # Abnormal2 Time Domain Feature
        TimeFeatureTest_Abnormal2[10*j+0, i] = np.max(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal2[10*j+1, i] = np.min(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal2[10*j+2, i] = np.mean(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal2[10*j+3, i] = rms(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal2[10*j+4, i] = np.var(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal2[10*j+5, i] = sp.skew(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal2[10*j+6, i] = sp.kurtosis(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal2[10*j+7, i] = np.max(temp_test_data2.iloc[:,j])/rms(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal2[10*j+8, i] = rms(temp_test_data2.iloc[:,j])/np.mean(temp_test_data2.iloc[:,j])
        TimeFeatureTest_Abnormal2[10*j+9, i] = np.max(temp_test_data2.iloc[:,j])/np.mean(temp_test_data2.iloc[:,j])
        
print(TimeFeatureTest_Normal.shape)
print(TimeFeatureTest_Abnormal1.shape)
print(TimeFeatureTest_Abnormal2.shape)

In [ ]:
#Frequency Domain 특징값 추출 (Wavelet Transform 기반)
FreqFeatureTest_Normal   = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfTestData))
FreqFeatureTest_Abnormal1 = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfTestData))
FreqFeatureTest_Abnormal2 = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfTestData))

for i in range(NoOfTestData):
    
    # 데이터 불러오기
    temp_test_path1 = './Test_Data/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_test_path2 = './Test_Data/Abnormal1_%d'%(i+1) # Abnormal1 데이터 파일 경로
    temp_test_path3 = './Test_Data/Abnormal2_%d'%(i+1) # Abnormal2 데이터 파일 경로
    temp_test_data1 = pd.read_csv(temp_test_path1 , sep=',' , header=None) # 임시 Normal 데이터
    temp_test_data2 = pd.read_csv(temp_test_path2 , sep=',' , header=None) # 임시 Abnormal1 데이터
    temp_test_data3 = pd.read_csv(temp_test_path3 , sep=',' , header=None) # 임시 Abnormal2 데이터
    Coef1      = pywt.wavedec(temp_test_data1, MotherWavelet, level=Level, axis=0)
    Coef2      = pywt.wavedec(temp_test_data2, MotherWavelet, level=Level, axis=0)
    Coef3      = pywt.wavedec(temp_test_data3, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in range(NoOfSensor):
        
        for k in np.arange(select):
            coef1 = Coef1[Level-k]
            coef2 = Coef2[Level-k]
            coef3 = Coef3[Level-k]
            
            # Normal Frequency Domain Feature
            FreqFeatureTest_Normal[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef1[:,j])
            FreqFeatureTest_Normal[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef1[:,j])
            FreqFeatureTest_Normal[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef1[:,j])
            FreqFeatureTest_Normal[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef1[:,j])
            FreqFeatureTest_Normal[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef1[:,j])
            FreqFeatureTest_Normal[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef1[:,j])
            FreqFeatureTest_Normal[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef1[:,j])
            FreqFeatureTest_Normal[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeatureTest_Normal[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef1[:,j])/np.mean(coef1[:,j])
            FreqFeatureTest_Normal[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef1[:,j])/np.mean(coef1[:,j])
            
            # Abnormal1 Frequency Domain Feature
            FreqFeatureTest_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef2[:,j])
            FreqFeatureTest_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef2[:,j])
            FreqFeatureTest_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef2[:,j])
            FreqFeatureTest_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef2[:,j])
            FreqFeatureTest_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef2[:,j])
            FreqFeatureTest_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef2[:,j])
            FreqFeatureTest_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef2[:,j])
            FreqFeatureTest_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef2[:,j])/rms(coef2[:,j])
            FreqFeatureTest_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef2[:,j])/np.mean(coef2[:,j])
            FreqFeatureTest_Abnormal1[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef2[:,j])/np.mean(coef2[:,j])
            
            # Abnormal2 Frequency Domain Feature
            FreqFeatureTest_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef3[:,j])
            FreqFeatureTest_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef3[:,j])
            FreqFeatureTest_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef3[:,j])
            FreqFeatureTest_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef3[:,j])
            FreqFeatureTest_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef3[:,j])
            FreqFeatureTest_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef3[:,j])
            FreqFeatureTest_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef3[:,j])
            FreqFeatureTest_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef3[:,j])/rms(coef3[:,j])
            FreqFeatureTest_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef3[:,j])/np.mean(coef3[:,j])
            FreqFeatureTest_Abnormal2[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef3[:,j])/np.mean(coef3[:,j])

print(FreqFeatureTest_Normal.shape)
print(FreqFeatureTest_Abnormal1.shape)
print(FreqFeatureTest_Abnormal2.shape)

In [ ]:
Test_Normal = np.concatenate([TimeFeatureTest_Normal,FreqFeatureTest_Normal] , axis=0)
Test_Abnormal1 = np.concatenate([TimeFeatureTest_Abnormal1,FreqFeatureTest_Abnormal1] , axis=0)
Test_Abnormal2 = np.concatenate([TimeFeatureTest_Abnormal2,FreqFeatureTest_Abnormal2] , axis=0)
Normal_TestData = pd.DataFrame(Test_Normal)
Abnormal1_TestData = pd.DataFrame(Test_Abnormal1)
Abnormal2_TestData = pd.DataFrame(Test_Abnormal2)

In [ ]:
Number = 20

Normal_SelectedTestFeatues   = np.zeros((Number,NoOfTestData))
Abnormal1_SelectedTestFeatues = np.zeros((Number,NoOfTestData))
Abnormal2_SelectedTestFeatues = np.zeros((Number,NoOfTestData))

for i in range(Number):
    
    index         = int(P_value_Rank.iloc[i,0])
    Normal_SelectedTestFeatues[i,:]   = Normal_TestData.iloc[index,:].values
    Abnormal1_SelectedTestFeatues[i,:] = Abnormal1_TestData.iloc[index,:].values
    Abnormal2_SelectedTestFeatues[i,:] = Abnormal2_TestData.iloc[index,:].values

# 정상, 고장 특징값 합치기    
TestFeatureSelected = pd.DataFrame(np.concatenate([Normal_SelectedTestFeatues, Abnormal1_SelectedTestFeatues, Abnormal2_SelectedTestFeatues] , axis=1))
TestFeatureSelected.shape

### 데이터 분할

In [ ]:
FeatureSelected = pd.DataFrame(np.transpose(FeatureSelected))
FeatureSelected.shape

In [ ]:
Training_All = FeatureSelected.iloc[:,:]
Training_All.shape

In [ ]:
NoOfTrainData = int(Training_All.shape[0]/3)
print(NoOfTrainData)

In [ ]:
Training_All_Label_forANN = np.zeros((NoOfTrainData*3,3), dtype=int)

Training_All_Label_forANN[:NoOfTrainData,0] = 1
Training_All_Label_forANN[NoOfTrainData:NoOfTrainData*2 , 1] = 1
Training_All_Label_forANN[NoOfTrainData*2:,2] = 1
Training_All_Label_forANN = pd.DataFrame(Training_All_Label_forANN)

Training_All_Label_forANN.shape

In [ ]:
TestFeatureSelected = pd.DataFrame(np.transpose(TestFeatureSelected))
TestFeatureSelected.shape

In [ ]:
Test_All = TestFeatureSelected.iloc[:,:]
Test_All.shape

In [ ]:
Test_All_Label_forANN = np.zeros((NoOfTestData*3,3), dtype=int)

Test_All_Label_forANN[:NoOfTestData,0] = 1
Test_All_Label_forANN[NoOfTestData:NoOfTestData*2 , 1] = 1
Test_All_Label_forANN[NoOfTestData*2:,2] = 1
Test_All_Label_forANN = pd.DataFrame(Test_All_Label_forANN)

Test_All_Label_forANN.shape

In [ ]:
Test_All_Label_forANN.to_csv('./Data/Test_Label_forANN', sep = ",", header = None, index = None)

### ANN hyper-parameter 찾기

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.random.set_seed(777)

In [ ]:
learningRate1  = 0.01
learningRate2  = 0.005
learningRate3  = 0.001

noOfNeuron1    = 4
noOfNeuron2    = 8
noOfNeuron3    = 16

iteration    = 300

In [ ]:
Data  = np.array(Training_All)
Label = np.array(Training_All_Label_forANN)

In [ ]:
Training_All  = np.array(Training_All)
Training_All_Label_forANN = np.array(Test_All_Label_forANN)
Test_All  = np.array(Test_All)
Test_All_Label_forANN = np.array(Test_All_Label_forANN)

### Model1 (learningRate  = 0.01, noOfNeuron    = 4)

In [ ]:
def ANN_model1(input_data):
    model1 = keras.Sequential()
    model1.add(keras.layers.Dense(units = noOfNeuron1,
                                 input_shape = (input_data.shape[1],), activation = 'relu'))    # Input  Layer
    model1.add(keras.layers.Dense(units = noOfNeuron1,                 activation = 'relu'))    # Hidden Layer 1
    model1.add(keras.layers.Dense(units = noOfNeuron1,                 activation = 'relu'))    # Hidden Layer 2
    model1.add(keras.layers.Dense(units = 3,                           activation = 'softmax')) # Output Layer
    
    
    
    model1.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate1),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model1

In [ ]:
tf.random.set_seed(777)    

model1 = ANN_model1(Training_All)
hist1 = model1.fit(Data, Label, epochs=iteration, verbose = 0)
Loss1, Accuracy1 = model1.evaluate(Test_All, Test_All_Label_forANN, verbose=0)

Accuracy1

### Model2 (learningRate  = 0.01, noOfNeuron    = 8)

In [ ]:
def ANN_model2(input_data):
    model2 = keras.Sequential()
    model2.add(keras.layers.Dense(units = noOfNeuron2,
                                 input_shape = (input_data.shape[1],), activation = 'relu'))    # Input  Layer
    model2.add(keras.layers.Dense(units = noOfNeuron2,                 activation = 'relu'))    # Hidden Layer 1
    model2.add(keras.layers.Dense(units = noOfNeuron2,                 activation = 'relu'))    # Hidden Layer 2
    model2.add(keras.layers.Dense(units = 3,                           activation = 'softmax')) # Output Layer
    
    
    
    model2.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate1),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model2

In [ ]:
tf.random.set_seed(777)    

model2 = ANN_model2(Training_All)
hist2 = model2.fit(Data, Label, epochs=iteration, verbose = 0)
Loss2, Accuracy2 = model2.evaluate(Test_All, Test_All_Label_forANN, verbose=0)

Accuracy2

### Model3 (learningRate  = 0.01, noOfNeuron    = 16)

In [ ]:
def ANN_model3(input_data):
    model3 = keras.Sequential()
    model3.add(keras.layers.Dense(units = noOfNeuron3,
                                 input_shape = (input_data.shape[1],), activation = 'relu'))    # Input  Layer
    model3.add(keras.layers.Dense(units = noOfNeuron3,                 activation = 'relu'))    # Hidden Layer 1
    model3.add(keras.layers.Dense(units = noOfNeuron3,                 activation = 'relu'))    # Hidden Layer 2
    model3.add(keras.layers.Dense(units = 3,                           activation = 'softmax')) # Output Layer
    
    
    
    model3.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate1),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model3

In [ ]:
tf.random.set_seed(777)    

model3 = ANN_model3(Training_All)
hist3 = model3.fit(Data, Label, epochs=iteration, verbose = 0)
Loss3, Accuracy3 = model3.evaluate(Test_All, Test_All_Label_forANN, verbose=0)

Accuracy3

### Model4 (learningRate  = 0.005, noOfNeuron    = 4)

In [ ]:
def ANN_model4(input_data):
    model4 = keras.Sequential()
    model4.add(keras.layers.Dense(units = noOfNeuron1,
                                 input_shape = (input_data.shape[1],), activation = 'relu'))    # Input  Layer
    model4.add(keras.layers.Dense(units = noOfNeuron1,                 activation = 'relu'))    # Hidden Layer 1
    model4.add(keras.layers.Dense(units = noOfNeuron1,                 activation = 'relu'))    # Hidden Layer 2
    model4.add(keras.layers.Dense(units = 3,                           activation = 'softmax')) # Output Layer
    
    
    
    model4.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate2),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model4

In [ ]:
tf.random.set_seed(777)    

model4 = ANN_model4(Training_All)
hist4 = model4.fit(Data, Label, epochs=iteration, verbose = 0)
Loss4, Accuracy4 = model4.evaluate(Test_All, Test_All_Label_forANN, verbose=0)

Accuracy4

### Model5 (learningRate  = 0.005, noOfNeuron    = 8)

In [ ]:
def ANN_model5(input_data):
    model5 = keras.Sequential()
    model5.add(keras.layers.Dense(units = noOfNeuron2,
                                 input_shape = (input_data.shape[1],), activation = 'relu'))    # Input  Layer
    model5.add(keras.layers.Dense(units = noOfNeuron2,                 activation = 'relu'))    # Hidden Layer 1
    model5.add(keras.layers.Dense(units = noOfNeuron2,                 activation = 'relu'))    # Hidden Layer 2
    model5.add(keras.layers.Dense(units = 3,                           activation = 'softmax')) # Output Layer
    
    
    
    model5.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate2),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model5

In [ ]:
tf.random.set_seed(777)    

model5 = ANN_model5(Training_All)
hist5 = model5.fit(Data, Label, epochs=iteration, verbose = 0)
Loss5, Accuracy5 = model5.evaluate(Test_All, Test_All_Label_forANN, verbose=0)

Accuracy5

### Model6 (learningRate  = 0.005, noOfNeuron    = 16)

In [ ]:
def ANN_model6(input_data):
    model6 = keras.Sequential()
    model6.add(keras.layers.Dense(units = noOfNeuron3,
                                 input_shape = (input_data.shape[1],), activation = 'relu'))    # Input  Layer
    model6.add(keras.layers.Dense(units = noOfNeuron3,                 activation = 'relu'))    # Hidden Layer 1
    model6.add(keras.layers.Dense(units = noOfNeuron3,                 activation = 'relu'))    # Hidden Layer 2
    model6.add(keras.layers.Dense(units = 3,                           activation = 'softmax')) # Output Layer
    
    
    
    model6.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate2),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model6

In [ ]:
tf.random.set_seed(777)    

model6 = ANN_model6(Training_All)
hist6 = model6.fit(Data, Label, epochs=iteration, verbose = 0)
Loss6, Accuracy6 = model6.evaluate(Test_All, Test_All_Label_forANN, verbose=0)

Accuracy6

### Model7 (learningRate  = 0.001, noOfNeuron    = 4)

In [ ]:
def ANN_model7(input_data):
    model7 = keras.Sequential()
    model7.add(keras.layers.Dense(units = noOfNeuron1,
                                 input_shape = (input_data.shape[1],), activation = 'relu'))    # Input  Layer
    model7.add(keras.layers.Dense(units = noOfNeuron1,                 activation = 'relu'))    # Hidden Layer 1
    model7.add(keras.layers.Dense(units = noOfNeuron1,                 activation = 'relu'))    # Hidden Layer 2
    model7.add(keras.layers.Dense(units = 3,                           activation = 'softmax')) # Output Layer
    
    
    
    model7.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate3),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model7

In [ ]:
tf.random.set_seed(777)    

model7 = ANN_model7(Training_All)
hist7 = model7.fit(Data, Label, epochs=iteration, verbose = 0)
Loss7, Accuracy7 = model7.evaluate(Test_All, Test_All_Label_forANN, verbose=0)

Accuracy7

### Model8 (learningRate  = 0.001, noOfNeuron    = 8)

In [ ]:
def ANN_model8(input_data):
    model8 = keras.Sequential()
    model8.add(keras.layers.Dense(units = noOfNeuron2,
                                 input_shape = (input_data.shape[1],), activation = 'relu'))    # Input  Layer
    model8.add(keras.layers.Dense(units = noOfNeuron2,                 activation = 'relu'))    # Hidden Layer 1
    model8.add(keras.layers.Dense(units = noOfNeuron2,                 activation = 'relu'))    # Hidden Layer 2
    model8.add(keras.layers.Dense(units = 3,                           activation = 'softmax')) # Output Layer
    
    
    
    model8.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate3),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model8

In [ ]:
tf.random.set_seed(777)    

model8 = ANN_model8(Training_All)
hist8 = model8.fit(Data, Label, epochs=iteration, verbose = 0)
Loss8, Accuracy8 = model8.evaluate(Test_All, Test_All_Label_forANN, verbose=0)

Accuracy8

### Model9 (learningRate  = 0.001, noOfNeuron    = 16)

In [ ]:
def ANN_model9(input_data):
    model9 = keras.Sequential()
    model9.add(keras.layers.Dense(units = noOfNeuron3,
                                 input_shape = (input_data.shape[1],), activation = 'relu'))    # Input  Layer
    model9.add(keras.layers.Dense(units = noOfNeuron3,                 activation = 'relu'))    # Hidden Layer 1
    model9.add(keras.layers.Dense(units = noOfNeuron3,                 activation = 'relu'))    # Hidden Layer 2
    model9.add(keras.layers.Dense(units = 3,                           activation = 'softmax')) # Output Layer
    
    
    
    model9.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate3),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model9

In [ ]:
tf.random.set_seed(777)    

model9 = ANN_model9(Training_All)
hist9 = model9.fit(Data, Label, epochs=iteration, verbose = 0)
Loss9, Accuracy9 = model9.evaluate(Test_All, Test_All_Label_forANN, verbose=0)

Accuracy9

### 각 모델 진단 정확도

In [ ]:
for i in range(9):
    s1 = "Accuracy%d = round(Accuracy%d , 2)"%(i+1,i+1)
    exec(s1)

Acc = [[Accuracy1, Accuracy2, Accuracy3], [Accuracy4, Accuracy5, Accuracy6], [Accuracy7, Accuracy8, Accuracy9]]
Acc_df = pd.DataFrame(Acc, index=['0.01', '0.005', '0.001'],columns= ['4', '8', '16'])
Acc_df

### 모델저장

In [ ]:
model6.save("./Data/ANN_Bestmodel.h5")